In [1]:
import nltk
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams, SearchParams
import uuid

/home/debmalya/.pyenv/versions/3.12.10/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download NLTK sentence tokenizer
nltk.download('punkt', quiet=True)

True

In [10]:
import nltk
nltk.data.path.append('/home/debmalya/nltk_data')

In [3]:
# Initialize embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
# Initialize Qdrant client (local or cloud - we chose local for now using docker - if issue, check port mapping)
client = QdrantClient(host="localhost", port=6333)

collection_name = "CSE291A-RAG-Project-Trial"

# Create collection
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

/tmp/ipykernel_118061/3285278501.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [11]:
# Step 1: Chunk text into sentences
def chunk_text(text):
    return nltk.sent_tokenize(text)

In [12]:
# Step 2: Embed and store chunks in Qdrant
def store_chunks(text):
    chunks = chunk_text(text)
    embeddings = model.encode(chunks)

    points = [
        PointStruct(
            id=str(uuid.uuid4()), 
            vector=embedding.tolist(), 
            payload={"text": chunk} # add other metadata that we might need to store (recency, etc..)
        ) for chunk, embedding in zip(chunks, embeddings)        
    ]
    
    client.upsert(collection_name=collection_name, points=points)
    print(f"Stored {len(points)} chunks in Qdrant vector store.")

In [13]:
# Step 3: Query Qdrant with semantic search
def search_chunks(query, top_k=5):
    query_vector = model.encode(query).tolist()
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k,
        search_params=SearchParams(hnsw_ef=128)
    )
    return [hit.payload["text"] for hit in results]

In [14]:
text = """
    Retrieval-Augmented Generation (RAG) is a powerful technique that combines external knowledge retrieval with language generation. 
    It allows models to access relevant information beyond their training data. 
    This improves factual accuracy and reduces hallucination.
""".strip()

store_chunks(text)

query = "How does RAG improve accuracy?"
results = search_chunks(query)

print("Relevant Chunks:")
for i, res in enumerate(results, 1):
    print(f"{i}. {res}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/debmalya/nltk_data'
    - '/home/debmalya/.pyenv/versions/3.12.10/nltk_data'
    - '/home/debmalya/.pyenv/versions/3.12.10/share/nltk_data'
    - '/home/debmalya/.pyenv/versions/3.12.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/debmalya/nltk_data'
**********************************************************************
